# 필수 프로그램 설치(Google Colab 에서 실행)

In [ ]:
!pip3 install selenium selenium-wire  # 셀레니움 설치
!pip3 install selenium
!apt install chromium-browser # 크롬 브라우저 설치
!apt install chromium-chromedriver # 크롬 드라이버 설치
!cp /usr/lib/chromium-browser/chromedriver /usr/bin # 크롬 드라이버 복사


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 7.9 MB/s 
     |████████████████████████████████| 239 kB 46.2 MB/s 
     |████████████████████████████████| 140 kB 45.1 MB/s 
     |████████████████████████████████| 384 kB 25.6 MB/s 
     |████████████████████████████████| 58 kB 1.6 MB/s 
     |████████████████████████████████| 57 kB 2.3 MB/s 
     |████████████████████████████████| 57 kB 2.5 MB/s 
     |████████████████████████████████| 2.5 MB 50.6 MB/s 
     |████████████████████████████████| 357 kB 55.4 MB/s 
     |████████████████████████████████| 4.0 MB 34.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

# 프로그램 로드

## 배터리 패키지 로드
배터리: 프로그램을 설치하면 기본적으로 같이 설치되는 패키지들.
python을 설치한 후 pip를 하지 않고도 바로 import 할 수 있는 패키지를 지칭한다.

In [ ]:
import time
import csv
from typing import List, NamedTuple, Tuple
import unicodedata
from random import randint

## 배터리 아닌 패키지 로드

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

import selenium
# from seleniumwire import webdriver
from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver import Firefox, FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

## 브라우저 생성 함수
1. 어떤 브라우저를 실행시킬 것인가?
2. headless 모드(화면 없이 작동)를 활성화 할 것인가?

In [ ]:
def get_browser():
    """브라우저 생성"""
    chrome_path = "/usr/bin/chromedriver"
    
    # Google Colab에서 실행시키기 위해서 옵션을 설정함
    # Windows에서 실행할 경우 알아서 설정하셔요
    options = webdriver.ChromeOptions()

    # 화면 표시 안함
    options.add_argument("headless")

    # 화면 크기
    options.add_argument('--windows-size=1920x1080')

    # ua 설정
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36')

    # 사생활 보호 모드로 실행
    # https://stackoverflow.com/questions/27630190/python-selenium-incognito-private-mode
    options.add_argument('--incognito')
    options.add_argument("--private")

    # 구글 콜랩을 위한 각종 에러 방지
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors')
    options.add_argument("--disable-popup-blocking")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    return webdriver.Chrome(chrome_path, options=options)

## 구글 드라이브와 연결한다

In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


## 데이터를 저장하는 함수

In [ ]:
def save_data(filename: str, data: list):
    df = pd.DataFrame([{"title": d["title"], "preview": "\n".join(d["preview"]).strip()} for d in data])
    df.to_json(f"/content/gdrive/My Drive/yes24_{filename}.json")
    #df.to_json(f"/content/gdrive/My Drive/yes24.latest.json")

# 페이지 파싱하는 코드

In [ ]:
'''# 전체 데이터를 담는 변수
data = []

# 이미 수집한 책인지 확인할수 있도록 제목만 저장하는 집합 변수
title_set = set()'''

'# 전체 데이터를 담는 변수\ndata = []\n\n# 이미 수집한 책인지 확인할수 있도록 제목만 저장하는 집합 변수\ntitle_set = set()'

In [ ]:
title_set = set()

## 베스트 목록에서 크롤링

In [ ]:
def parse_page_from_best(browser, data):
    """베스트 목록에서 데이터를 추출한다."""
    # 리스트 추출
    elements = browser.find_elements(By.CSS_SELECTOR, "#category_layout > tbody:nth-child(1) > tr")

    # 목록이 없으면 끝까지 크롤링 완료했다고 여김
    if not elements:
        return False

    else:
        for elem in tqdm(elements[::2]):
            #---------- Selenium.START ----------
            # 제목 크롤링
            title_elem = elem.find_element(By.CSS_SELECTOR, "td.goodsTxtInfo > p > a")
            title = title_elem.get_attribute('innerHTML')
            #---------- Selenium.END ----------


            #---------- BeautifulSoup.START ----------
            soup = BeautifulSoup(elem.get_attribute('innerHTML'), "html.parser")

            # 미리보기 버튼이 있는지 확인함
            is_preview_availalbe = bool(soup.select_one('img[alt="이북 미리보기"]'))
            # 없으며 패스
            if not is_preview_availalbe:
                data.append({"title": title, "preview": []})
                continue
            #---------- BeautifulSoup.END ----------

            # 이미 수집한 제목은 넘김
            if title in title_set:
                continue
            else:
                title_set.add(title)

            #---------- Selenium.START ----------
            # 미리보기 열기
            preview_button = elem.find_element(By.CSS_SELECTOR, "td:nth-child(2) > div:nth-child(1) > a:nth-child(2)")
            js_code = preview_button.get_attribute('href')
            browser.execute_script(js_code.lstrip("javascript:"))

            # 미리보기 열릴 때까지 2초 대기
            time.sleep(2)

            # 팝업창으로 이동
            parent_window_id = browser.current_window_handle
            popup_window_id = browser.window_handles[1]
            browser.switch_to.window(popup_window_id)

            # 창 크기 조절
            browser.set_window_size(640, 320)
            #---------- Selenium.END ----------


            #---------- BeautifulSoup.START ----------
            preview_content = []
            try:
                # 미리보기 내용 가져옴
                preview_soup = BeautifulSoup(browser.page_source, "html.parser")
                for elem in preview_soup.select_one("div.viewer_window").select("li.chapter > p.txt"):
                    preview_content.append(elem.text.replace("\xa0", ""))
            except:
                print("미리보기 수집 실패:", title)
            #---------- BeautifulSoup.END ----------
            
            # 팝업창 닫기
            browser.close()
            
            # 원래 창으로 복귀
            browser.switch_to.window(parent_window_id)

            # 데이터를 리스트에 저장함
            data.append({"title": title, "preview": preview_content})
        return True

## 일반 소설 목록에서 크롤링

In [ ]:
def parse_page_from_normal(browser, data):
    """일반 목록에서 데이터를 추출한다."""
    # 리스트 추출
    elements = browser.find_elements(By.CSS_SELECTOR, "#category_layout > ul.clearfix > li")

    # 목록이 없으면 끝까지 크롤링 완료했다고 여김
    if not elements:
        return False

    else:
        for elem in tqdm(elements):
            #---------- Selenium.START ----------
            # 제목 크롤링
            title_elem = elem.find_element(By.CSS_SELECTOR, "div.goods_info > div.goods_name > a")
            title = title_elem.get_attribute('innerHTML')
            #---------- Selenium.END ----------


            #---------- BeautifulSoup.START ----------
            soup = BeautifulSoup(elem.get_attribute('innerHTML'), "html.parser")

            # 미리보기 버튼이 있는지 확인함
            is_preview_availalbe = bool(soup.select_one('a.btnC.btn_preview > span > em'))
            # 없으며 패스
            if not is_preview_availalbe:
                data.append({"title": title, "preview": []})
                continue
            #---------- BeautifulSoup.END ----------

            # 이미 수집한 제목은 넘김
            if title in title_set:
                continue
            else:
                title_set.add(title)

            #---------- Selenium.START ----------
            # 미리보기 열기
            preview_button = elem.find_element(By.CSS_SELECTOR, "a.btnC.btn_preview")
            js_code = preview_button.get_attribute('href')
            browser.execute_script(js_code.lstrip("javascript:"))

            # 미리보기 열릴 때까지 2초 대기
            time.sleep(2)

            # 팝업창으로 이동
            parent_window_id = browser.current_window_handle
            popup_window_id = browser.window_handles[1]
            browser.switch_to.window(popup_window_id)

            # 창 크기 조절
            browser.set_window_size(640, 320)
            #---------- Selenium.END ----------


            #---------- BeautifulSoup.START ----------
            preview_content = []
            try:
                # 미리보기 내용 가져옴
                preview_soup = BeautifulSoup(browser.page_source, "html.parser")
                for div in soup.find_all("div", {'class':'author'}):
                    div.decompose()
                
                for elem in preview_soup.select_one("div.viewer_window").select("li.chapter > p"):
                    preview_content.append(elem.text.replace("\xa0", ""))
            except:
                print("미리보기 수집 실패:", title)
            #---------- BeautifulSoup.END ----------
            
            # 팝업창 닫기
            browser.close()
            
            # 원래 창으로 복귀
            browser.switch_to.window(parent_window_id)

            # 데이터를 리스트에 저장함
            data.append({"title": title, "preview": preview_content})
        return True

## 베스트 목록 크롤링 작동 코드

In [ ]:
def crawling_at_steady_list(min_page: int = 0, max_page: int = 1000):
    data = []
    for i in range(min_page, max_page):
        url = f"http://www.yes24.com/24/category/bestseller?CategoryNumber=017001045&sumgb=03&pagenumber={i + 1}"
        print(i+1, "페이지 크롤링 中...")
        
        # 브라우저 열면서 크롤링 실시
        browser = get_browser()
        browser.get(url)
        
        # 과부하 방지를 위해 3초 휴식
        time.sleep(3)
        
        # 데이터 크롤링
        parse_page_from_best(browser, data)
        
        try:
            browser.close()
        except:
            pass
        try:
            print("미리보기 보유 비율:", (len([1 for d in data if d['preview'].strip()]) / len(data)) * 100, "%")
        except:
            pass

    print('steady셀러 저장 중')        
    save_data("steady", data)

## 월별 베스트 목록 크롤링 작동 코드

In [ ]:
def crawling_at_monthly_best_list(year: int = 2014, month: int = 5, min_page: int = 0, max_page: int = 1000):
    data = []
    
    # 얼마나 남은지 모르므로 while
    while True:
        
        # 년월 계산
        if month == 12:
            year += 1
            month = 1
        else:
            month += 1
        
        # 미래인가?
        if year == 2022 and month == 7:
            # 종료
            break
        
        # url 생성
        url = f"http://www.yes24.com/24/category/bestseller?CategoryNumber=017001045&sumgb=09&year={year}&month={month}"

        
        for i in range(min_page, max_page):
            
            if i:
                url = url + f"&pagenumber={i + 1}"
            print(year, "년", month, "월", i, "페이지 크롤링 中...")
            
            # 브라우저 열면서 크롤링 실시
            browser = get_browser()
            browser.get(url)
            
            # 과부하 방지를 위해 3초 휴식
            time.sleep(3)
            
            try:
                # 데이터 크롤링
                if not parse_page_from_best(browser, data):
                    break
            except:
                pass
            finally:
                try:
                    browser.close()
                except:
                    pass

        save_data(f"monthly_best", data)

## 일반 목록 크롤링 작동 코드

In [ ]:
def crawling_at_normal_list(category_id: str, name: str, min_page: int = 0, max_page = 1000):
    data = []
    
    # 크롤링 할 페이지 지정
    for i in range(min_page, max_page):
        
        # 조건에 맞는 url 생성
        # FetchSize=40 40개를 로딩함
        # A0=2  성인용 제외
        # pagenumber=n 크롤링할 페이지 번호
        url = f"http://www.yes24.com/24/Category/Display/{category_id}?FetchSize=40&AO=2&pagenumber={i + 1}"
        print(name, i+1, "페이지 크롤링 中...")
        
        # 브라우저 열면서 크롤링 실시
        browser = get_browser()
        browser.get(url)
        
        # 과부하 방지를 위해 3초 휴식
        time.sleep(3)
        
        # 데이터 크롤링
        r = parse_page_from_normal(browser,data)
        
        try:
            browser.close()
        except:
            pass
        
        if (i+1)%10 == 0:
          print(f'{i+1} 페이지까지 데이터 저장 중')
          save_data(f"normal_{name}_{i+1}", data)
          data = []
          

        # 만약, 리스트가 비어 있었다면 크롤링 종료
        if not r:
            break
    
    if data != []:
      print(f'마지막 페이지까지 데이터 저장 중')
      save_data(f'normal_{name}_last', data)
        

    

# 크롤링 대상 목록

In [ ]:
target_list = [
               ("017001045006", "korean", 0, 100),
               ("017001045007", "english", 0, 100),
               ("017001045016", "detector", 0, 100),
               ("017001045017", "sf", 0, 40),
               ("017001045022", "history", 0, 33),
               ("017001045018", "family", 0, 10),
               ("017001045019", "love", 0, 17),
               ("017001045020", "fairy_tail_for_older", 0, 10),
               ("017001045021", "from_movie", 0, 10),
               ("017001045024", "scenario", 0, 10)
]

# 크롤링

In [ ]:
title_set = set()

In [ ]:
# yes24 스테디 셀러 목록 크롤링
crawling_at_steady_list(0, 42)



1 페이지 크롤링 中...


<ipython-input-4-c7f19e1eb555>:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Chrome(chrome_path, options=options)


  0%|          | 0/20 [00:00<?, ?it/s]

2 페이지 크롤링 中...


  0%|          | 0/20 [00:00<?, ?it/s]

...

41 페이지 크롤링 中...


  0%|          | 0/20 [00:00<?, ?it/s]

42 페이지 크롤링 中...


  0%|          | 0/19 [00:00<?, ?it/s]

steady셀러 저장 중


In [ ]:
# yes24 월별 베스트 목록 크롤링
#crawling_at_monthly_best_list(2022, 5, 0, 1)

In [ ]:
'''for i in target_list[0]:
    category_id = i[0]
    category_name = i[1]
    min_page = i[2]
    max_page = i[3]
    crawling_at_normal_list(category_id, category_name, min_page, max_page)'''

TypeError: ignored

In [ ]:
target = target_list[0]
category_id = target[0]
category_name = target[1]
min_page = 30
max_page = target[3]
crawling_at_normal_list(category_id, category_name, min_page, max_page)

korean 31 페이지 크롤링 中...


<ipython-input-4-c7f19e1eb555>:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Chrome(chrome_path, options=options)


  0%|          | 0/40 [00:00<?, ?it/s]

korean 32 페이지 크롤링 中...


  0%|          | 0/40 [00:00<?, ?it/s]

...

korean 99 페이지 크롤링 中...


  0%|          | 0/40 [00:00<?, ?it/s]

korean 100 페이지 크롤링 中...


  0%|          | 0/40 [00:00<?, ?it/s]

100 페이지까지 데이터 저장 중


In [ ]:
target = target_list[1]
category_id = target[0]
category_name = target[1]
min_page = 80
max_page = target[3]
crawling_at_normal_list(category_id, category_name, min_page, max_page)

english 81 페이지 크롤링 中...


<ipython-input-4-c7f19e1eb555>:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Chrome(chrome_path, options=options)


  0%|          | 0/40 [00:00<?, ?it/s]

english 82 페이지 크롤링 中...


  0%|          | 0/40 [00:00<?, ?it/s]

...

english 99 페이지 크롤링 中...


  0%|          | 0/40 [00:00<?, ?it/s]

english 100 페이지 크롤링 中...


  0%|          | 0/40 [00:00<?, ?it/s]

100 페이지까지 데이터 저장 중
